In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import TimeSeriesSplit

## QUANTILE REGRESSION

In [5]:
data = pd.read_csv('non_nan_4.csv', )
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [6]:
def create_lags(data, tickers, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs, lagged_tickers

In [7]:
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period = [1, 2, 3]

data, lagged_covs, lagged_tickers = create_lags(data, tickers, covs, period)

In [8]:
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,NVO_lag3,NVS_lag1,NVS_lag2,NVS_lag3,PFE_lag1,PFE_lag2,PFE_lag3,ROG_lag1,ROG_lag2,ROG_lag3
0,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,...,2.220031,2.559423,6.420237,3.838386,15.213674,14.101954,-11.226228,7.370518,6.922258,54.440789
1,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,...,8.390897,5.169310,2.559423,6.420237,5.638019,15.213674,14.101954,-8.163265,7.370518,6.922258
2,2000-07-01,4.0,172.700,6.097273,-0.653855,-1.338453,-8.064479,-11.373392,-8.650271,4.005041,...,-0.097663,8.474599,5.169310,2.559423,8.076012,5.638019,15.213674,13.131313,-8.163265,7.370518
3,2000-08-01,4.1,172.700,6.054000,0.795710,6.534124,6.578935,3.131245,-1.192113,-29.708189,...,20.863985,-3.593737,8.474599,5.169310,-9.635421,8.076012,5.638019,2.142857,13.131313,-8.163265
4,2000-09-01,3.9,173.600,5.826087,0.249670,-5.723123,15.363525,8.458631,2.496165,11.482185,...,2.813690,-1.944901,-3.593737,8.474599,-0.540360,-9.635421,8.076012,-2.447552,2.142857,13.131313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,9.244201,1.295221,10.390734,6.735944,3.090302,7.690670,6.375682,9.446071,-53.665066,-2.707956
273,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,14.479974,-0.110227,1.295221,10.390734,-13.817335,3.090302,7.690670,16.968326,9.446071,-53.665066
274,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,8.619586,-7.172811,-0.110227,1.295221,-7.286115,-13.817335,3.090302,5.451681,16.968326,9.446071
275,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,2.541749,9.367574,-7.172811,-0.110227,0.566924,-7.286115,-13.817335,11.025813,5.451681,16.968326


### Defining tick_loss function

In [9]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

### 1-step-ahead VaR: 1 month

In [10]:
def rolling_1m_forecast(data, dates, ticker, alpha):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/12

    return t_loss  

In [11]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']

In [12]:
for t in tickers:
    loss = rolling_1m_forecast(data, dates1m, t, 0.05)
    print(f'{t}: {loss}')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN: 0.9188584337762072


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarul

BMY: 0.5640033475614191


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ: 0.77330935386026
LLY: 0.38019830316269093


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK: 0.7009559022485864


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO: 1.5186452512956368
NVS: 1.0024847249001598
PFE: 1.04254996710358
ROG: 4.841139747957954


In [13]:
d='2022-05-01'
train_df = data[data.Date <= d]
test_df = data[data.Date > d]
test_df = test_df.reset_index(drop=True)

X_train = train_df.drop(columns=['NVS', 'Date'])
y_train = train_df['NVS']

X_test = test_df.drop(columns=['NVS', 'Date'])
y_test = test_df['NVS']

quantreg = sm.QuantReg(y_train, X_train)
quantreg_results = quantreg.fit(q=0.05)

VaR_forecast = quantreg_results.predict(X_test)

In [14]:
print(type(VaR_forecast))
VaR_forecast

<class 'pandas.core.series.Series'>


0     -3.403468
1     -5.824003
2    -11.405746
3     -4.605742
4     -0.408990
5     -1.440417
6     -4.489651
7    -19.651154
8    -14.154497
9     -1.559925
10    -7.813986
11   -14.589936
dtype: float64

### 3-step-ahead VaR: 3 months

In [15]:
def rolling_3m_forecast(data, dates, ticker, alpha):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/10

    return t_loss  

In [16]:
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']

In [17]:
for t in tickers:
    loss = rolling_3m_forecast(data, dates3m, t, 0.05)
    print(f'{t}: {loss}')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN: 1.0173735756126532


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY: 0.6042375901941596


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ: 0.8822143952936482
LLY: 0.2887010225306154


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK: 0.8026611776677084


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO: 1.2700829669248423
NVS: 1.0521797777370427
PFE: 1.1026535485891293
ROG: 4.817047425074099


### 6-step-ahead VaR: 6 months

In [18]:
def rolling_6m_forecast(data, dates, ticker, alpha):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/7

    return t_loss  

In [19]:
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [20]:
for t in tickers:
    loss = rolling_6m_forecast(data, dates6m, t, 0.05)
    print(f'{t}: {loss}')

AZN: 1.0848589678006804


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY: 0.7024537971230377


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ: 0.36379446167053425
LLY: 0.3014703641791335


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK: 0.9898418026424336


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO: 1.561466083479276
NVS: 1.2643260324118286
PFE: 0.20048866423216744
ROG: 6.409458408721784


### 9-step-ahead VaR: 9 months

In [21]:
def rolling_9m_forecast(data, dates, ticker, alpha):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/4

    return t_loss  

In [22]:
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']

In [23]:
for t in tickers:
    loss = rolling_9m_forecast(data, dates9m, t, 0.05)
    print(f'{t}: {loss}')

AZN: 0.951457564712485


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY: 0.7584554524298539


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ: 0.3454076778567384
LLY: 0.36781202171984734


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK: 0.7117757290884085
NVO: 1.8917314978646262
NVS: 1.9059747560683196
PFE: 0.16116027454173176
ROG: 1.043805133808879


### 12-step-ahead VaR: 12 months

In [24]:
def rolling_12m_forecast(data, dates, ticker, alpha):

    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)

        r = y_test[0]
        loss = tick_loss(alpha, r, VaR_forecast)

        t_loss += loss

    t_loss = t_loss/1

    return t_loss  

In [25]:
dates12m = ['2022-05-01']

In [26]:
for t in tickers:
    loss = rolling_12m_forecast(data, dates12m, t, 0.05)
    print(f'{t}: {loss}')

AZN: 0.8499547839654987


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY: 1.1665385900696297
JNJ: 0.45816672947207915
LLY: 0.20257883379957967


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK: 0.5724836643794061
NVO: 1.0353095049570613
NVS: 3.4628816251498193
PFE: 0.21259675008240803
ROG: 0.5644800968365005


### ABBV case

In [35]:
data2 = pd.read_csv('monthly_data.csv')

In [38]:
data2 = data2[['Date', 'ABBV']]
data2

,Date,ABBV
0,2000-01-01,NaN
1,2000-02-01,NaN
2,2000-03-01,NaN
3,2000-04-01,NaN
4,2000-05-01,NaN
...,...,...
278,2023-03-01,3.554254
279,2023-04-01,-5.176628
280,2023-05-01,-7.868480
281,2023-06-01,NaN


In [39]:
data2 = data2.dropna()
data2

,Date,ABBV
158,2013-03-01,10.455062
159,2013-04-01,12.922975
160,2013-05-01,-6.438455
161,2013-06-01,-3.162299
162,2013-07-01,10.014486
...,...,...
276,2023-01-01,-8.576205
277,2023-02-01,5.152609
278,2023-03-01,3.554254
279,2023-04-01,-5.176628


In [42]:
data = pd.read_csv('non_nan_4.csv')

In [43]:
merged_df = data2.merge(data, on='Date', how='inner')
merged_df

,Date,ABBV,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2013-03-01,10.455062,7.5,232.282,1.984211,0.397212,3.337507,14.582205,11.414669,7.978126,4.851792,3.440210,-7.714279,9.067089,5.443922,-0.188640,-0.655,-0.2,0.069449
1,2013-04-01,12.922975,7.6,231.797,1.957500,0.085246,2.374537,3.881563,-3.568848,4.538222,-2.482836,7.359867,11.497419,3.537347,0.727651,-10.457790,-0.485,0.1,-0.026711
2,2013-05-01,-6.438455,7.5,231.893,1.759091,-0.190605,2.361004,-1.290451,16.816781,-1.231989,-4.008663,-0.638313,-8.792401,-2.711502,-6.329562,8.513136,0.096,-0.1,-0.198409
3,2013-06-01,-3.162299,7.5,232.445,1.928182,0.620501,-1.853781,-7.707318,-2.868894,2.762464,-6.788137,-0.535372,-3.805087,-1.463219,3.722003,2.269287,0.552,0.0,0.169091
4,2013-07-01,10.014486,7.3,232.900,2.300000,0.210359,5.724173,7.230424,-3.244595,8.898141,8.122938,4.655361,9.001745,1.272809,4.355584,17.540151,0.455,-0.2,0.371818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023-01-01,-8.576205,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
119,2023-02-01,5.152609,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
120,2023-03-01,3.554254,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
121,2023-04-01,-5.176628,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [46]:
tickers = ['ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period = [1, 2, 3]

merged_data, lagged_covs, lagged_tickers = create_lags(merged_df, tickers, covs, period)

In [48]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_1m_forecast(merged_data, dates1m, t, 0.05)
        print(f'{t}: {loss}')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ABBV: 1.8091799216980309


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [49]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_3m_forecast(merged_data, dates3m, t, 0.05)
        print(f'{t}: {loss}')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ABBV: 1.5865023386040358


In [50]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_6m_forecast(merged_data, dates6m, t, 0.05)
        print(f'{t}: {loss}')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ABBV: 2.058112006066971


In [51]:
for t in tickers:
    if t == 'ABBV':
        loss = rolling_9m_forecast(merged_data, dates9m, t, 0.05)
        print(f'{t}: {loss}')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ABBV: 2.879691122201558


### QUANTILE REGRESSION VS BASELINE

In [53]:
import pandas as pd

data_1m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [0.416, 0.587, 0.725, 0.620, 0.518, 0.609, 0.735, 0.588, 0.899, 3.772],
    'Quantile Regression': [1.002, 1.809, 0.919, 0.604, 0.773, 0.380, 0.701, 1.519, 1.043, 4.841]
}

df_1M = pd.DataFrame(data_1m)

df_1M['diff'] = df_1M['Historical Quantile']-df_1M['Quantile Regression']
df_1M

,Ticker,Historical Quantile,Quantile Regression,diff
0,NVS,0.416,1.002,-0.586
1,ABBV,0.587,1.809,-1.222
2,AZN,0.725,0.919,-0.194
3,BMY,0.620,0.604,0.016
4,JNJ,0.518,0.773,-0.255
5,LLY,0.609,0.380,0.229
6,MRK,0.735,0.701,0.034
7,NVO,0.588,1.519,-0.931
8,PFE,0.899,1.043,-0.144
9,ROG,3.772,4.841,-1.069


In [54]:
data_3m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [1.299, 1.731, 2.164, 1.873, 1.443, 1.807, 2.215, 1.730, 2.394, 12.951],
    'Quantile Regression': [1.052, 1.587, 1.017, 0.604, 0.882, 0.289, 0.803, 1.270, 1.103, 4.817]
}

df_3M = pd.DataFrame(data_3m)

df_3M['diff'] = df_3M['Historical Quantile']-df_3M['Quantile Regression']
df_3M


,Ticker,Historical Quantile,Quantile Regression,diff
0,NVS,1.299,1.052,0.247
1,ABBV,1.731,1.587,0.144
2,AZN,2.164,1.017,1.147
3,BMY,1.873,0.604,1.269
4,JNJ,1.443,0.882,0.561
5,LLY,1.807,0.289,1.518
6,MRK,2.215,0.803,1.412
7,NVO,1.730,1.270,0.460
8,PFE,2.394,1.103,1.291
9,ROG,12.951,4.817,8.134


In [55]:
data_6m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [2.731, 3.991, 5.725, 4.406, 3.895, 4.285, 5.170, 3.677, 4.776, 26.300],
    'Quantile Regression': [1.264, 2.058, 1.085, 0.702, 0.363, 0.301, 0.990, 1.561, 1.264, 6.409]
}

df_6M = pd.DataFrame(data_6m)

df_6M['diff'] = df_6M['Historical Quantile']-df_6M['Quantile Regression']
df_6M

,Ticker,Historical Quantile,Quantile Regression,diff
0,NVS,2.731,1.264,1.467
1,ABBV,3.991,2.058,1.933
2,AZN,5.725,1.085,4.640
3,BMY,4.406,0.702,3.704
4,JNJ,3.895,0.363,3.532
5,LLY,4.285,0.301,3.984
6,MRK,5.170,0.990,4.180
7,NVO,3.677,1.561,2.116
8,PFE,4.776,1.264,3.512
9,ROG,26.300,6.409,19.891


In [56]:
data_9m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [2.731, 3.991, 5.725, 4.406, 3.895, 4.285, 5.170, 3.677, 4.776, 26.300],
    'Quantile Regression': [1.906, 2.880, 0.951, 0.758, 0.345, 0.368, 0.712, 1.892, 0.161, 1.044]
}

df_9M = pd.DataFrame(data_9m)
df_9M['diff'] = df_9M['Historical Quantile']-df_9M['Quantile Regression']
df_9M

,Ticker,Historical Quantile,Quantile Regression,diff
0,NVS,2.731,1.906,0.825
1,ABBV,3.991,2.880,1.111
2,AZN,5.725,0.951,4.774
3,BMY,4.406,0.758,3.648
4,JNJ,3.895,0.345,3.550
5,LLY,4.285,0.368,3.917
6,MRK,5.170,0.712,4.458
7,NVO,3.677,1.892,1.785
8,PFE,4.776,0.161,4.615
9,ROG,26.300,1.044,25.256


In [60]:
data_12m = {
    'Ticker': ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG'],
    'Historical Quantile': [5.275, 7.109, 8.276, 7.670, 5.860, 7.081, 8.847, 7.067, 10.229, 45.244],
    'Quantile Regression': [3.462, 0.770, 0.850, 1.167, 0.458, 0.203, 0.572, 1.035, 0.212, 0.564],
}

df_12M = pd.DataFrame(data_12m)
df_12M['diff'] = df_12M['Historical Quantile']-df_12M['Quantile Regression']
df_12M

,Ticker,Historical Quantile,Quantile Regression,diff
0,NVS,5.275,3.462,1.813
1,ABBV,7.109,0.770,6.339
2,AZN,8.276,0.850,7.426
3,BMY,7.670,1.167,6.503
4,JNJ,5.860,0.458,5.402
5,LLY,7.081,0.203,6.878
6,MRK,8.847,0.572,8.275
7,NVO,7.067,1.035,6.032
8,PFE,10.229,0.212,10.017
9,ROG,45.244,0.564,44.680


10

In [65]:
df_1M['diff'].head(9).sum()

-3.053

In [70]:
df_1M['diff'].sum()

-4.122

In [67]:
df_3M['diff'].head(9).sum()

8.049000000000001

In [71]:
df_3M['diff'].sum()

16.183

In [68]:
df_6M['diff'].head(9).sum()

29.067999999999998

In [72]:
df_6M['diff'].sum()

48.959

In [69]:
df_9M['diff'].head(9).sum()

28.683

In [73]:
df_9M['diff'].sum()

53.939

In [66]:
df_12M['diff'].head(9).sum()

58.685

In [74]:
df_12M['diff'].sum()

103.36500000000001